# Goal

* Flagellome analysis based on FlaPro results: HMP2 IBD dataset.


# Var

In [ ]:
work_dir = '/ebio/abt3_projects2/human_fla_profiling/atyakht/human-fla-profiling/'
PROJECT_TAG = "IBD"

AVAILABLE_OMICS = c("MGX", "MTX")
#AVAILABLE_OMICS = c("MTX")

DO_MTX_MGX_ratio = TRUE

# - scenario for the Component-base analysis, 3 options:
#SCENARIO_COMPON = "MGX" # - metagenomics (MGX, meta-DNA); OR
#SCENARIO_COMPON = "MTX" # - metatranscriptomics (MTX, meta-RNA)
SCENARIO_COMPON = "MTX_MGX_ratio" # - metatranscriptomics (MTX, meta-RNA) / metagenomics (MGX, meta-DNA) ratio

# - scenario for the Compositional analysis (Nearest Balance), 2 options:
#SCENARIO_NB = "MGX"
SCENARIO_NB = "MTX"

# flagellome profiles data and meta-data files
flapro_file = list()
coverage_file = list()

# DB v. C4 (Feb 2025):
flapro_file[["MGX"]] = file.path(work_dir, "data/IBD/merged_realcounts_MGX.txt")
coverage_file[["MGX"]] = file.path(work_dir, "data/IBD/full_hmp_mgx_readcount.txt")

flapro_file[["MTX"]] = file.path(work_dir, "data/IBD/merged_realcounts_MTX.txt")
coverage_file[["MTX"]] = file.path(work_dir, "data/IBD/full_hmp_mtx_readcount.txt")

# meta-data
meta_samples_file = list()
meta_samples_file[["MGX"]] = file.path(work_dir, "data/IBD/final_metadata_mgx-24-09-24.txt")
meta_samples_file[["MTX"]] = file.path(work_dir, "data/IBD/final_metadata_mtx-24-09-24.txt")
hmp_metadata_original_file = file.path(work_dir, "data/IBD/hmp2_metadata_2018-08-20.csv")

# meta-data
meta_fla_file = file.path(work_dir, "data/taxonomy_cluster_repr_c4-pred3.tsv")

In [ ]:
# sample filtering params
MIN_FLA_READS_PER_SAMPLE = 100 #50 #30 # pre-prevalence filtering
MIN_FLA_READS_PER_SAMPLE_ROUND_2 = 30 # post-prevalence filtering

# filtering params for the relative abundance (flapro_rel)
REL_AB_PREVALENCE_CUTOFF = 25 #30
REL_AB_ABUND_CUTOFF = 0 #0.0001

# filtering params for NB (flapro), as well as its normalized version (flapro_perc)
PREVALENCE_CUTOFF = 30 #10
ABUND_CUTOFF_PERC = 0 #0.0001 #0.01 #0.005

# flapro_rel: coefficient of sigma for outlier detection
# (set it to some high value like 1E6 to disable outlier filtering)
REL_N_SIGMA_REL_OUTLIERS = 3 #0.5
# number of features (Fla) to show in the biplot
REL_N_FEATURES_BIPLOT = 5 #10

# colors used
FLA_CLASSES_COLORS = c("not_defined" = "#888888", "active" = "#5050ff", "silent" = "#ce3d32", "evader" = "cyan", "mixed" = "black")
FLA_CLASSES_SHAPES = c("not_defined" = 4, "active" = 1, "silent" = 3, "evader" = 4, "mixed" = 10)

# option: dedicated analysis of the Fla experimentally profiled
EXPLORE_EXPERIMENTAL_FLA = FALSE

# additionally adjust features for factors (by collecting the residuals) - 
# - for the Component-based analysis (SCENARIO_COMPON) - for the viz purposes, does not affect the LM:
ADD_ADJUST_FOR_FACTORS_COMPON = FALSE
# - for the Compositional analysis (SCENARIO_NB) - for the viz purposes as well the produced NB values:
ADD_ADJUST_FOR_FACTORS_NB = TRUE
# adjust for what:
ADD_ADJUST_FOR = "Age + Sex"

# Adjust this factor as needed
BIPLOT_ARROW_SCALING = c("MGX" = 5e3, "MTX" = 5e4, "MTX_MGX_ratio" = 2e3)
# add text labels to the biplot samples?
BIPLOT_LABELS_SAMPLES = FALSE

# repeated measures present? (e.g. multiple time points per subject)
# - for the Component-based analysis (_COMPON), to use lmer instead of lm:
REPEAT_MEAS_COMPON = TRUE
# - for the Compositional analysis (_NB), to generate 1-sample-per subject splits instead of common splits:
REPEAT_MEAS_NB = TRUE
# name of the repeated measures factor (used if flags above enabled)
REPEAT_MEAS_FACTOR = "Participant_ID"

In [ ]:
# Nearest Balance parameters
# cross-validation parameters:
# threshold for reproducibility of Nearest Balance
reproducibility_threshold = 0.8 #0.9

# setting for leave-1-out
n_sim = 100  # number of cross-validation simulations
train_prop = 0.67 # proportion of samples to use for training (except for the case of repeated measures where it's defined customly)

RANDOM_SEED = 123 # for reproducibility

In [ ]:
# clin. groups to be included in comparisons
GROUPS_TO_COMPARE = c("HC", "UC", "CD")
#GROUPS_TO_COMPARE = c("HC", "UC")
#GROUPS_TO_COMPARE = c("HC", "CD")

#sel_factor = "Group"
sel_factor = "DiseaseScore"

STAT_PLOT_CMP = list(c("HC", "UC"), c("HC", "CD"), c("UC", "CD"))
COHORT_COLORS = c("HC" = "#008ea0", "UC" = "#ff6f00", "CD" = "#c71000")

In [ ]:
rel_model_formula = paste0(sel_factor, " + Age + (1|Participant_ID)")

# initial formula for PERMANOVA (many factors)
init_permanova_formula = paste0(sel_factor, " + Age + Sex")

# LM formula for Nearest Balance - factors selected based on adonis2 above
lm_nb_formula = paste0(sel_factor, " + Age")

# select 1 factor for which we seek Nearest Balance - 
# along with the name of coef that LM give to it (might be different for categorical factors).
if(sel_factor == "Group") {
    testthat::test_that("two cohorts are selected", {
        testthat::expect_equal(length(GROUPS_TO_COMPARE), 2)
    })
    #sel_factor_coef = paste0("Group", sort(GROUPS_TO_COMPARE, decreasing = TRUE)[1])
    sel_factor_coef = paste0("Group", GROUPS_TO_COMPARE[length(GROUPS_TO_COMPARE)])
} else {
    sel_factor_coef = "DiseaseScore"
}

#sel_factor = "Age"
#sel_factor_coef = "Age"

In [ ]:
# number of permutations for PERMANOVA
N_PERMANOVA = 999 # for testing only
#N_PERMANOVA = 9999

In [ ]:
# Set the number of CPU cores to use:
# in the "parallel" lib
num_rparallel_cores = 40

# Init

In [ ]:
library(readxl)
library(NearestBalance)
library(zCompositions)
library(reshape2)
library(selbal)
library(LeyLabRMisc)
library(ggpubr)
library(data.table)
library(ggplot2)
library(tibble)
library(vegan)
library(foreach)
library(doParallel)
library(lme4) 
library(textshape)
library(mgcv)
library(MASS)
library(ggsci)
library(cluster)
library(tidyr)
library(readr)
library(broom)
library(lmerTest)
library(broom.mixed)
library(furrr)
library(testthat)
library(ggrepel)
library(pheatmap)
library(dplyr)
library(purrr)

df.dims(5)

In [ ]:
# in furrr
future::plan(multicore)

In [ ]:
source(file.path(work_dir, "../r_helper_lib", "nb_helpers.R"))
source(file.path(work_dir, "../r_helper_lib", "functions_mb.R"))

In [ ]:
set.seed(RANDOM_SEED)  # Set seed for reproducibility

In [ ]:
# suffix for some output files
GROUPS_SUFFIX = paste(GROUPS_TO_COMPARE, collapse = "_")

# create project-specific output folder if not exists
PROJ_OUTPUT_DIR = file.path(work_dir, "out", PROJECT_TAG)
if (!dir.exists(PROJ_OUTPUT_DIR)) {
  dir.create(PROJ_OUTPUT_DIR, recursive = TRUE)
}

# config-specific output subfolder
tmp = file.path(PROJ_OUTPUT_DIR, paste0(GROUPS_SUFFIX))
if (!dir.exists(tmp)) {
  dir.create(tmp, recursive = TRUE)
}
CFG_OUTPUT_DIR = file.path(tmp, paste0(sel_factor_coef))
if (!dir.exists(CFG_OUTPUT_DIR)) {
  dir.create(CFG_OUTPUT_DIR, recursive = TRUE)
}

In [ ]:
# output multisheet XLS file with the statistical tests results -- component-based approach
out_stat_compon_xlsx_file = file.path(CFG_OUTPUT_DIR, paste0("stats_", SCENARIO_COMPON, ".xlsx"))

# Load

## features

In [ ]:
flapro = lapply(flapro_file, function(x) {
    read_tsv(x, col_names = TRUE) %>% 
        pivot_longer(cols = -c(Family), names_to = "Sample", values_to = "Abundance") %>% 
        rename(FlaCluster_Rep = Family)
})

## meta - features

In [ ]:
meta_fla = read_tsv(meta_fla_file, col_names = TRUE)
meta_fla

meta_fla %>% select(Flagellin_ID) %>% distinct() %>% nrow()
meta_fla %>% select(Cluster_c4_representative) %>% distinct() %>% nrow()

meta_fla = meta_fla %>% 
    mutate(num_fla_per_cluster = n(), .by = Cluster_c4_representative) 

meta_fla %>% select(Cluster_c4_representative, num_fla_per_cluster) %>% 
    distinct() %>%
    arrange(desc(num_fla_per_cluster))

In [ ]:
# replace NA with "not_defined", to obtain true table() output
meta_fla = meta_fla %>%     
    mutate(Predicted = ifelse(is.na(Predicted_v3) | Predicted_v3 == "not_checked", "not_defined", Predicted_v3)) %>%
    select(-Predicted_v3) %>% 
    mutate(Experimental = ifelse(is.na(Experimental) | Experimental == "not_checked", "not_defined", Experimental))    

In [ ]:
# meta_fla: make Cluster_Pred_v which summarizes Predicted_v by Cluster_c4_representative in a way that if the value is the same, it's left; otherwise, it's assigned "mixed"
meta_fla = meta_fla %>%     
    mutate(Cluster_Pred = ifelse(n_distinct(Predicted) == 1, first(Predicted), "mixed"), .by = Cluster_c4_representative) %>% 
    mutate(Cluster_Exp = ifelse(n_distinct(Experimental) == 1, first(Experimental), "mixed"), .by = Cluster_c4_representative) %>% 
    # make Cluster_Species by concatenating all distinct Species , per Cluster_c4_representative
    mutate(Cluster_Species = paste(unique(Species), collapse = ";"), .by = Cluster_c4_representative) %>% 
    mutate(Cluster_Genus = paste(unique(Genus), collapse = ";"), .by = Cluster_c4_representative) %>%
    mutate(Cluster_Family = paste(unique(Family), collapse = ";"), .by = Cluster_c4_representative)

In [ ]:
meta_fla %>% select(Cluster_Pred) %>% table()
meta_fla %>% select(Cluster_Exp) %>% table()

## sample coverage

In [ ]:
sample_coverage = lapply(coverage_file, function(x) {
    read_tsv(x, col_names = FALSE) %>% 
        rename(Sample = "X1", Reads1 = "X2") %>% 
        distinct()
})

## [ps] meta - samples

In [ ]:
meta_samples = lapply(AVAILABLE_OMICS, function(x) {
    read_tsv(meta_samples_file[[x]], col_names = TRUE) %>%     
        rename(Participant_ID = "Participant.ID", Internal_BioSample_ID = "sample_alias_nodif", Sample = "run_accession", Group = "diagnosis")
}) %>% setNames(AVAILABLE_OMICS)

In [ ]:
hmp_metadata_original = read_csv(hmp_metadata_original_file, col_names = TRUE) %>% 
    rename(Participant_ID = "Participant ID", Age = "consent_age", Sex = "sex")        
hmp_metadata_original

# Preprocess

## [ps] rename feature and sample coverage tables

In [ ]:
flapro = lapply(AVAILABLE_OMICS, function(x) {
    flapro[[x]] %>% inner_join(meta_samples[[x]], by = "Sample") %>%         
        mutate(Sample = Internal_BioSample_ID) %>% 
        select(FlaCluster_Rep, Sample, Abundance)    
}) %>% setNames(AVAILABLE_OMICS)

In [ ]:
sample_coverage = lapply(AVAILABLE_OMICS, function(x) {
    sample_coverage[[x]] %>% inner_join(meta_samples[[x]], by = "Sample") %>% 
        mutate(Sample = Internal_BioSample_ID) %>% 
        select(Sample, Reads1)    
}) %>% setNames(AVAILABLE_OMICS)

## [ps] metadata - samples

In [ ]:
# merge the 2 meta_samples into 1
meta_samples = lapply(AVAILABLE_OMICS, function(x) {
    meta_samples[[x]] %>% 
        mutate(Sample = Internal_BioSample_ID) %>% 
        select(Sample, Participant_ID, Group, week_num, visit_num)
}) %>% setNames(AVAILABLE_OMICS)
meta_samples

In [ ]:
meta_samples = do.call(rbind, meta_samples[AVAILABLE_OMICS]) %>% distinct() 
meta_samples

In [ ]:
# order Participant_ID by Group for viz purposes
meta_samples = meta_samples %>% 
    mutate(Participant_ID = factor(Participant_ID, 
                                 levels = unique(Participant_ID[order(Group)])))
meta_samples

In [ ]:
# meta_samples: plot lines for weeks_num per Participant_ID, arranged by diagnosis
p.dims(15, 3)
meta_samples %>% 
    ggplot(aes(x = Participant_ID, y = week_num, color = Group)) +
    geom_point() +
    geom_line() +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +
    scale_color_futurama()

In [ ]:
meta_samples = meta_samples %>%    
    select(Sample, Participant_ID, Group, week_num)
meta_samples

In [ ]:
# add Sex and Age from the large meta table
meta_samples = meta_samples %>% inner_join(hmp_metadata_original %>% select(Participant_ID, Age, Sex) %>% distinct(), by = "Participant_ID") 

In [ ]:
# in meta_samples Group, replace values HC - with 0_HC, UC - with 1_UC, CD - with 2_CD
meta_samples = meta_samples %>%     
    mutate(Group = str_replace(Group, "nonIBD", "HC")) %>%
    # make Diagnosis a factor, with levels ordered as HC, UC, CD
    mutate(Group = factor(Group, levels = c("HC", "UC", "CD")))
meta_samples

In [ ]:
# leave only those who have sex
nrow(meta_samples %>% filter(is.na(Sex)))
meta_samples = meta_samples %>% filter(!is.na(Sex))

# and age
nrow(meta_samples %>% filter(is.na(Age)))
meta_samples = meta_samples %>% filter(!is.na(Age))

nrow(meta_samples)

In [ ]:
meta_samples %>% select(Participant_ID) %>% distinct() %>% nrow()
meta_samples %>% select(Participant_ID, Group) %>% distinct() %>% select(Group) %>% table()

### [ps] Create derivative factor/s

In [ ]:
# DiseaseScore is a quantitative approximation for a severity of disease (considering UC is normally less severe than CD; with HC = 0)
meta_samples = meta_samples %>% 			
	mutate(DiseaseScore = ifelse(Group == "HC", 0, ifelse(Group == "UC", 1, ifelse(Group == "CD", 2, NA))))	
meta_samples

## metadata - flagellins

In [ ]:
# for those species that are not defined, we will use the genus
# (and before - the same for genus & family)
meta_fla = meta_fla %>% 
    mutate(Genus = ifelse(is.na(Genus), paste0("uncl. ", Family), Genus))

meta_fla = meta_fla %>% 
    mutate(Genus = ifelse(Genus == "-", paste0(Family, " gen."), Genus)) %>% 
    mutate(Species = ifelse(Species == "-", paste0(Genus, " spp."), Species))

In [ ]:
# add a letter prefix to the Cluster
meta_fla = meta_fla %>% mutate(FlaCluster = paste0("FC_", Cluster_c4_representative)) %>% 
    select(-Cluster_c4_representative)
meta_fla

In [ ]:
meta_fla %>% select(Flagellin_ID) %>% distinct() %>% nrow()
meta_fla %>% select(FlaCluster) %>% distinct() %>% nrow()

In [ ]:
# prepare a per-FlaCluster version of meta_fla (with distinct())
meta_fla_clus = meta_fla %>%  
    #Content_of_the_cluster	num_fla_per_cluster	Cluster_Pred	Cluster_Exp	Cluster_Species	Cluster_Genus	Cluster_Family	FlaCluster
    select(FlaCluster, Cluster_Pred, Cluster_Exp, Cluster_Family, Cluster_Genus, Cluster_Species) %>%     
    distinct()
meta_fla_clus

## sample coverage

## feature matrices

In [ ]:
flapro = lapply(flapro, function(x) {
    x %>% mutate(FlaCluster = paste0("FC_", FlaCluster_Rep)) %>%
        select(-FlaCluster_Rep)
})

In [ ]:
# leave FlaProfiles only for those samples who have meta-data        
flapro = lapply(AVAILABLE_OMICS, function(x) {
    print(x)
    y = flapro[[x]]    
    print(y %>% unique_n('samples', Sample))
    print(y %>% unique_n('features', FlaCluster))
    res = y %>% filter(Sample %in% meta_samples$Sample)
    print(y %>% unique_n('samples present in meta', Sample))
    print(y %>% unique_n('features', FlaCluster))
    res
}) %>% setNames(AVAILABLE_OMICS)

In [ ]:
### for saving: remove zero Fla and make a filtered meta_fla accordingly

In [ ]:
flapro_no_zeros = lapply(AVAILABLE_OMICS, function(x) {
    flapro[[x]] %>% filter(Abundance > 0)
}) %>% setNames(AVAILABLE_OMICS)
flapro_no_zeros

flapro_no_zeros_mat = lapply(AVAILABLE_OMICS, function(x) {
    flapro_no_zeros[[x]] %>% 
        pivot_wider(names_from = Sample, values_from = Abundance, values_fill = 0) %>% 
        column_to_rownames("FlaCluster") %>% 
        as.matrix()
}) %>% setNames(AVAILABLE_OMICS)
flapro_no_zeros_mat

In [ ]:
meta_fla_no_zeros = lapply(AVAILABLE_OMICS, function(x) {
    meta_fla %>% 
    select(FlaCluster, Cluster_Family, Cluster_Genus, Cluster_Species, Cluster_Pred, Cluster_Exp, Content_of_the_cluster) %>% 
    distinct() %>% 
    inner_join(flapro_no_zeros[[x]] %>% select(FlaCluster) %>% distinct(), by = "FlaCluster")
}) %>% setNames(AVAILABLE_OMICS)
meta_fla_no_zeros

### identify the samples with 0 or very low Fla counts

In [ ]:
flapro_stats = lapply(AVAILABLE_OMICS, function(x) {
    y = flapro[[x]]
    res = y %>% 
	    mutate(counts_per_sample = as.double(sum(Abundance)), .by = "Sample") %>%     
        select(Sample, counts_per_sample) %>% distinct() %>% arrange(desc(counts_per_sample))

    # histogram
    p.dims(20,3)
    plot(res %>% ggplot(aes(counts_per_sample)) + geom_histogram(bins = 150) + 
        ylab("Number of samples") +
        xlab("Number of reads per sample") +
        ggtitle(x) +
        theme_minimal())
    res
}) %>% setNames(AVAILABLE_OMICS)

In [ ]:
flapro_stats = lapply(flapro_stats, function(x) {
    x %>% inner_join(meta_samples, by = "Sample")
})

In [ ]:
flapro = lapply(flapro, function(x) {
	res = x %>% 
		group_by(Sample) %>% 
		mutate(counts_per_sample = as.double(sum(Abundance))) %>% 
		# for component-based branch, do NOT drop the samples with 0 or very low Fla counts (that would affect case-control!)
		###filter(counts_per_sample >= MIN_FLA_READS_PER_SAMPLE) %>% 
		ungroup() %>% 
		arrange(desc(counts_per_sample)) %>% 
		rename(feature = "FlaCluster", value = "Abundance")
	print(x %>% unique_n('samples', Sample))
	res
})
flapro

In [ ]:
lapply(flapro_stats, function(x) {	
    x %>% select(Sample) %>% distinct() %>% nrow()
})

In [ ]:
# save the list of samples with 0 or very low Fla counts
samples_with_low_fla = lapply(flapro_stats, function(x) {	
	x %>% filter(counts_per_sample < MIN_FLA_READS_PER_SAMPLE) %>% select(Sample)
})
samples_with_low_fla

In [ ]:
# 1: percentage of Fla calculated to the total Fla reads (a helper for NB branch)
flapro_perc = lapply(flapro, function(x) {
    x %>% 
        group_by(Sample) %>%     
        # for null-flagellome samples, assign 0 to Abundance_perc
        mutate(value = ifelse(value == 0, 0, 100.0*value/counts_per_sample)) %>%         
        select(-counts_per_sample) %>% 
        ungroup()        
})
flapro_perc

In [ ]:
# 2: relative abundance of Fla calculated relative to the metagenome coverage, not total Fla reads (used in component-based branch)
# Its sum per sample - not 100%!
flapro_rel = lapply(AVAILABLE_OMICS, function(x) {
    y = flapro[[x]]
    y %>% 
        inner_join(sample_coverage[[x]], by = "Sample") %>% 
        mutate(value = 1e8 * value/Reads1) %>% 
        arrange(feature, desc(value)) %>% 
        select(-counts_per_sample, -Reads1)
        
}) %>% setNames(AVAILABLE_OMICS)
flapro_rel

In [ ]:
flapro = lapply(flapro, function(x) {
    x %>% select(-counts_per_sample) %>% ungroup()
})
#flapro

#### heatmap of the major Fla (as per flapro_rel)

In [ ]:
flapro_rel_no_zeros_mat = lapply(AVAILABLE_OMICS, function(x) {
    flapro_rel[[x]] %>% inner_join(meta_fla_no_zeros[[x]] %>% select(FlaCluster), by = c("feature" = "FlaCluster")) %>% 
        pivot_wider(names_from = Sample, values_from = value) %>% 
        column_to_rownames("feature") %>% 
        as.matrix()
}) %>% setNames(AVAILABLE_OMICS)
flapro_rel_no_zeros_mat

In [ ]:
for(x in AVAILABLE_OMICS) {
    y = flapro_rel_no_zeros_mat[[x]]   
    p.dims(9, 5)
    pheatmap(t(y),
        #cluster_rows = FALSE, cluster_cols = FALSE, 
        cluster_rows = TRUE, cluster_cols = TRUE, 
        show_rownames = TRUE, show_colnames = TRUE, fontsize = 8, 
        fontsize_number = 10, fontsize_row = 7, fontsize_col = 7, number_color = "black",        
        #angle_col = 45,
        # margin around the heatmap 20
        margins = c(20, 20),        
        #color = colorRampPalette(c("blue", "white", "#ff9090", "red"))(4),
    #   legend = FALSE,
    #   annotation_row = feature_subclasses,  # Add column annotation
    #   annotation_names_row = FALSE,        
    #   annotation_colors = ann_colors,  # Assign colors to subclasses
        border_color = "white", 
        main = x
    )
}

#### save intermediate results to files

In [ ]:
for(x in AVAILABLE_OMICS) {
    flapro_rel_no_zeros_dump = 
        meta_fla_no_zeros[[x]] %>% inner_join(flapro_rel[[x]] %>%
            arrange(Sample) %>% 
            pivot_wider(names_from = Sample, values_from = value), by = c("FlaCluster" = "feature"))
    flapro_rel_no_zeros_dump

    flapro_no_zeros_dump = 
        meta_fla_no_zeros[[x]] %>% inner_join(flapro[[x]] %>%
            arrange(Sample) %>% 
            pivot_wider(names_from = Sample, values_from = value), by = c("FlaCluster" = "feature"))
    flapro_no_zeros_dump

    # save 
    write_tsv(flapro_rel_no_zeros_dump, file.path(PROJ_OUTPUT_DIR, paste0("flapro_rel_no_zeros_", x, ".tsv")))
    write_tsv(flapro_no_zeros_dump, file.path(PROJ_OUTPUT_DIR, paste0("flapro_counts_no_zeros_", x, ".tsv")))
}

### flapro_rel: derive the MTX/MGX ratio and add it to this list

In [ ]:
if(DO_MTX_MGX_ratio) {   
    flapro_rel[["MTX_MGX_ratio"]] = flapro_rel[["MGX"]] %>% rename(Abundance_rel_MGX = value) %>% 
        inner_join(flapro_rel[["MTX"]] %>% rename(Abundance_rel_MTX = value), by = c("Sample", "feature")) %>% 
        # below are filtering steps that have a large effect:    
        # 1: remove the features encoded in a few taxa

        filter(Abundance_rel_MGX > 0) %>% 

        ## (don't need this, as the step 1 will do) 2: remove the features with very low MGX or MTX levels
        #filter(Abundance_rel_MTX + Abundance_rel_MGX > 1e-10) %>% 
        mutate(Abundance_rel = Abundance_rel_MTX/Abundance_rel_MGX) %>% 
        select(Sample, feature, Abundance_rel) %>% 
        rename(value = "Abundance_rel")     
    
    # make the table full - for correct calculation of feature prevalence
    flapro_rel[["MTX_MGX_ratio"]] = flapro_rel[["MTX_MGX_ratio"]] %>% 
        pivot_wider(names_from = "feature", values_from = "value", values_fill = 0) %>%
        pivot_longer(cols = -Sample, names_to = "feature", values_to = "value") 

    print(flapro_rel[["MTX_MGX_ratio"]])
}

### Fla alpha diversity

In [ ]:
if(SCENARIO_COMPON != "MTX_MGX_ratio") {
    # split flapro by Cluster_Pred
    list_flapro_by_Cluster_Pred = flapro[[SCENARIO_COMPON]] %>%
        inner_join(meta_fla_clus, by = c("feature" = "FlaCluster")) %>%         
        group_split(Cluster_Pred)
    names(list_flapro_by_Cluster_Pred) = lapply(list_flapro_by_Cluster_Pred, function(x) {
        x %>% select(Cluster_Pred) %>% distinct() %>% pull()
    })
    list_flapro_by_Cluster_Pred = lapply(list_flapro_by_Cluster_Pred, function(x) { 
        x %>% select(Sample, value, feature)
    })
    #list_flapro_by_Cluster_Pred

    # add flapro_alpha to the list under the name "all"
    list_flapro_by_Cluster_Pred_and_all = c(list_flapro_by_Cluster_Pred, list("all" = flapro[[SCENARIO_COMPON]]))
    #list_flapro_by_Cluster_Pred_and_all
}

In [ ]:
if(SCENARIO_COMPON != "MTX_MGX_ratio") {
    list_flapro_alpha = lapply(names(list_flapro_by_Cluster_Pred_and_all), function(x) {
        y = list_flapro_by_Cluster_Pred_and_all[[x]]

        for_psq_otu_table = y %>% 
            pivot_wider(names_from = Sample, values_from = value) %>%
            column_to_rownames("feature") %>% 
            as.matrix() %>% 
            phyloseq::otu_table(taxa_are_rows = TRUE)
        #for_psq_otu_table

        for_psq_tax_table = meta_fla_clus %>% 
            column_to_rownames("FlaCluster") %>% 
            as.matrix() %>% 
            phyloseq::tax_table()
        #for_psq_tax_table

        for_psq_sample_table = meta_samples %>% 
            column_to_rownames("Sample") %>% 
            phyloseq::sample_data()
        #for_psq_sample_table

        # create phyloseq object
        psq = phyloseq::phyloseq(for_psq_otu_table, for_psq_tax_table, for_psq_sample_table)
        #psq

        # compute the Fla alpha diversity
        flapro_alpha = phyloseq::estimate_richness(psq, split = TRUE, measures = c("Observed", "Chao1", "Shannon")) %>% 
            rownames_to_column("Sample") %>% 
            select(-se.chao1) %>% 
            pivot_longer(cols = -Sample, names_to = "feature", values_to = "value")
        #print(flapro_alpha)

        ## save to file
        #write_tsv(flapro_alpha %>% pivot_wider(names_from = feature, values_from = value), file.path(PROJ_OUTPUT_DIR, paste0("flapro_alpha_", SCENARIO_COMPON, ".tsv")))

        flapro_alpha
    }) %>% setNames(names(list_flapro_by_Cluster_Pred_and_all))
}

In [ ]:
if(SCENARIO_COMPON != "MTX_MGX_ratio") {
    list_flapro_alpha = lapply(names(list_flapro_by_Cluster_Pred_and_all), function(x) {
        y = list_flapro_alpha[[x]]
        y %>% mutate(feature = paste0(feature, "_F_", x)) 
    }) %>% setNames(names(list_flapro_by_Cluster_Pred_and_all))
    #list_flapro_alpha
    # rbind the list into one tibble
    flapro_alpha = bind_rows(list_flapro_alpha)
    print(flapro_alpha)
}

In [ ]:
if(SCENARIO_COMPON != "MTX_MGX_ratio") {
    #  TODO thuink of what to save and under which name
    ## save to file
    ##write_tsv(flapro_alpha %>% pivot_wider(names_from = feature, values_from = value), file.path(PROJ_OUTPUT_DIR, paste0("flapro_alpha_", SCENARIO_COMPON, ".tsv")))
}

### next thing

In [ ]:
# for flapro and flapro_perc, but not for flapro_rel - filter out the samples with low total Fla
flapro = lapply(AVAILABLE_OMICS, function(x) {
    y = flapro[[x]]
    res = y %>% filter(!(Sample %in% samples_with_low_fla[[x]]$Sample))
    print(res %>% select(Sample) %>% distinct() %>% nrow())
    res
}) %>% setNames(AVAILABLE_OMICS)

flapro_perc = lapply(AVAILABLE_OMICS, function(x) {
    y = flapro_perc[[x]]
    y %>% filter(!(Sample %in% samples_with_low_fla[[x]]$Sample))
}) %>% setNames(AVAILABLE_OMICS)

### prevalence filtering (flapro & flapro_perc - one way, flapro_rel -> flapro_rel_MAJ - another!)

#### - flapro & flapro_perc

In [ ]:
# check the long tables are full (includes the same numbers of zeros as if it were wide) before computing the prevalence
lapply(flapro_perc, check_full, arg_feature_col = "feature")

In [ ]:
flapro_perc_w_prev = lapply(flapro_perc, function(x) {
	x %>%
		mutate(prev = sum(value > ABUND_CUTOFF_PERC) / length(value) * 100, 
				.by=c(feature)) %>% 
		select(-value, -Sample) %>%
		distinct() %>% 
		arrange(desc(prev))
})
flapro_perc_w_prev

In [ ]:
# plot a barplot of the flagellins prevalence arranged by the prevalence desc
#p.dims(15, 10)
p.dims(5, 8)
tt = flapro_perc_w_prev[[SCENARIO_NB]] %>% filter(prev > PREVALENCE_CUTOFF)
tt %>% 
    #ggplot(aes(x = reorder(FlaCluster, prev), y = prev, fill = FlaCluster)) + #Family)) + 
    
    inner_join(meta_fla_clus, by = c("feature" = "FlaCluster")) %>% 
        ggplot(aes(x = reorder(feature, prev), y = prev, fill = Cluster_Pred)) +     

        geom_bar(stat = "identity") + #, color = "black", size = 0.1) + 
        #scale_fill_igv() +
        scale_fill_manual(values = FLA_CLASSES_COLORS) +    

        # ! The below commented line does NOT work properly    
        ## label y axis by Family    
        #scale_x_discrete(labels = tt$Family) +     

        theme_bw() + theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
        coord_flip() +
        labs(x = "Flagellin cluster", y = "Prevalence (%)") +
        # smaller axis text
        theme(axis.text.x = element_text(size = 6), axis.text.y = element_text(size = 6)) +
        # legend - top
        theme(legend.position = "top") +
        # smaller legend
        theme(legend.text = element_text(size = 7), legend.title = element_text(size = 7)) +
        theme(legend.key.size = unit(0.5, "cm")) + 
        ggtitle(paste0("flapro_perc: ", SCENARIO_NB))

    #flapro_perc_w_prev %>% ggplot(aes(prev)) + geom_histogram(bins = 150) + theme_minimal()

In [ ]:
# FLAPRO_PERC: filtering low prevalence
flapro_perc = lapply(flapro_perc, function(x) {  
    print(x %>% unique_n('features before filtering', feature))
    res = x %>% 
        mutate(prev = sum(value > ABUND_CUTOFF_PERC) / length(value) * 100, 
                .by=c(feature)) %>%
        filter(prev > PREVALENCE_CUTOFF) %>%
        select(-prev)
    print(res %>% unique_n('features after filtering', feature))
    res
})

In [ ]:
# filter FLAPRO the same way as FLAPRO_PERC
flapro = lapply(AVAILABLE_OMICS, function(x) {  	
	flapro[[x]] %>% semi_join(flapro_perc[[x]], by = "feature")
}) %>% setNames(AVAILABLE_OMICS)

#### extra round of filtering: drop zero samples that might have appeared after the prevalence filtering

In [ ]:
flapro_stats2 = lapply(AVAILABLE_OMICS, function(x) {
    flapro[[x]] %>% 
	    mutate(counts_per_sample = as.double(sum(value)), .by = "Sample") %>%     
        select(Sample, counts_per_sample) %>% distinct() %>% arrange(desc(counts_per_sample))
}) %>% setNames(AVAILABLE_OMICS)

In [ ]:
samples_with_0_fla2 = lapply(flapro_stats2, function(x) {	
	x %>% filter(counts_per_sample < MIN_FLA_READS_PER_SAMPLE_ROUND_2) %>% select(Sample)
})
samples_with_0_fla2

In [ ]:
# flapro and flapro_perc: round 2 filter out the samples with low total Fla
flapro = lapply(AVAILABLE_OMICS, function(x) {
    flapro[[x]] %>% filter(!(Sample %in% samples_with_0_fla2[[x]]$Sample))    
}) %>% setNames(AVAILABLE_OMICS)

flapro_perc = lapply(AVAILABLE_OMICS, function(x) {
    flapro_perc[[x]] %>% filter(!(Sample %in% samples_with_0_fla2[[x]]$Sample))
}) %>% setNames(AVAILABLE_OMICS)

#### - flapro_rel ---> flapro_rel_maj

In [ ]:
flapro_rel[[SCENARIO_COMPON]] %>% select(feature) %>% distinct() %>% nrow()

In [ ]:
flapro_rel_w_prev = 
	flapro_rel[[SCENARIO_COMPON]] %>%		
		mutate(prev = sum(value > REL_AB_ABUND_CUTOFF) / length(value) * 100, 		
				.by=c(feature)) %>% 
		select(-value, -Sample) %>%
		distinct() %>% 
		arrange(desc(prev))
flapro_rel_w_prev

In [ ]:
# plot hist of flapro_rel_w_prev
p.dims(8, 2)
flapro_rel_w_prev %>% 
    ggplot(aes(prev)) + geom_histogram(bins = 150) + theme_minimal() +
    xlab("Prevalence (%)") +
    ylab("Number of features") +
    ggtitle(SCENARIO_COMPON) +
    # smaller axis text
    theme(axis.text.x = element_text(size = 6), axis.text.y = element_text(size = 6)) +
    # legend - top
    theme(legend.position = "top") +
    # smaller legend
    theme(legend.text = element_text(size = 7), legend.title = element_text(size = 7)) +
    theme(legend.key.size = unit(0.5, "cm")) + 
    scale_x_continuous(limits=c(0,100)) +
    scale_y_continuous(limits=c(0,50))

In [ ]:
# (OFF) for the ratio - additionally require high prevalence by MTX
if(FALSE) {
	if(SCENARIO_COMPON == "MTX_MGX_ratio") {
		prev_by_mtx = flapro_rel[["MTX"]] %>%
			mutate(prev_mtx = sum(value > REL_AB_ABUND_CUTOFF) / length(value) * 100, 
					.by=c(feature)) %>% 
			select(-value, -Sample) %>%
			distinct() %>% 

			filter(prev_mtx > REL_AB_PREVALENCE_CUTOFF) %>%

			arrange(desc(prev_mtx))
		
		print(prev_by_mtx)
		
		flapro_rel_w_prev = flapro_rel_w_prev %>% semi_join(prev_by_mtx, by = "feature")
	}

	flapro_rel_w_prev
}

In [ ]:
p.dims(5, 8)
tt = flapro_rel_w_prev %>% filter(prev > REL_AB_PREVALENCE_CUTOFF)
tt %>% 
    inner_join(meta_fla_clus, by = c("feature" = "FlaCluster")) %>% 
    ggplot(aes(x = reorder(feature, prev), y = prev, fill = Cluster_Pred)) +     

    geom_bar(stat = "identity") + #, color = "black", size = 0.1) + 
    scale_fill_manual(values = FLA_CLASSES_COLORS) +    
    theme_bw() + theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
    coord_flip() +
    labs(x = "Flagellin cluster", y = "Prevalence (%)") +
    # smaller axis text
    theme(axis.text.x = element_text(size = 6), axis.text.y = element_text(size = 6)) +
    # legend - top
    theme(legend.position = "top") +
    # smaller legend
    theme(legend.text = element_text(size = 7), legend.title = element_text(size = 7)) +
    theme(legend.key.size = unit(0.5, "cm")) + 
    ggtitle(paste0("flapro_rel: ", SCENARIO_COMPON))    

In [ ]:
# Filter flapro_rel using its own prevalence and abundance cutoffs, save the filtered as flapro_rel_maj
flapro_rel_maj = 
    flapro_rel_w_prev %>%     
    filter(prev > REL_AB_PREVALENCE_CUTOFF) %>%    
    select(-prev)
flapro_rel_maj %>% unique_n('features after filtering', feature)

In [ ]:
flapro_rel_maj = flapro_rel_maj %>% inner_join(flapro_rel[[SCENARIO_COMPON]], by = "feature")
flapro_rel_maj

In [ ]:
flapro_rel_maj %>% inner_join(meta_fla_clus, by = c("feature" = "FlaCluster")) %>% 
    select(feature, Cluster_Pred) %>% 
    distinct() %>% 
    select(Cluster_Pred) %>% 
    table()

In [ ]:
# do log transformation of the fla profiles
flapro_rel_maj_log = flapro_rel_maj %>% 
    mutate(value = log10(value + 1))
flapro_rel_maj_log

### prep-s for NB: init countData, for the selected omic (in line with flapro and flapro_perc)

In [ ]:
# creating count table
countData = 
    flapro[[SCENARIO_NB]] %>%
    rename('sample' = Sample) %>%
    select(sample, feature, value) %>%
    # only leave the samples that are in the meta
    semi_join(meta_samples, by = c("sample" = "Sample")) %>%
    pivot_wider(names_from = "sample", values_from = "value", values_fill=0) %>%
    # convert countData to a data frame with name moved to rownames
    column_to_rownames("feature") %>%
    as.data.frame

# sort columns and rows alphabetically
countData = countData[sort(rownames(countData)), sort(colnames(countData))]
countData

In [ ]:
lapply(AVAILABLE_OMICS, function(x) {
    y = flapro[[x]]
    res = y %>% 
	    mutate(counts_per_sample = as.double(sum(value)), .by = "Sample") %>%     
        select(Sample, counts_per_sample) %>% distinct() %>% arrange(desc(counts_per_sample))
    
    res %>% filter(counts_per_sample == 0)
}) %>% setNames(AVAILABLE_OMICS)

In [ ]:
# version of meta data table for the Compositional analysis
meta_samples_NB = meta_samples %>% semi_join(flapro[[SCENARIO_NB]], by = "Sample") %>% arrange(Sample)
meta_samples_NB

In [ ]:
test_that("flapro and meta fit", {
    expect_equal(colnames(countData), meta_samples_NB %>% select(Sample) %>% pull())    
})

# Analysis

## [ps] Non-microbiome basics

In [ ]:
# NONE

## Component-wise (flapro_REL)
over metagenome-coverage normalized relative abundance 

In [ ]:
flapro_rel[[SCENARIO_COMPON]] %>% inner_join(meta_samples, by = "Sample") %>% 
    select(Sample, Group) %>%
    distinct() %>%
    select(Group) %>%
    table()

### summarized Fla measures per sample

In [ ]:
# compute total *relative* fla counts per sample
flapro_rel_total_all = flapro_rel[[SCENARIO_COMPON]] %>% 
        group_by(Sample) %>% 
        summarise(sum_rel_abund = sum(value)) %>% 
        ungroup() %>%        
        rename(value = "sum_rel_abund") %>% 
        mutate(feature = "sum_rel_abund_F_all") %>%         
        #inner_join(meta_samples, by = "Sample") %>%      
        arrange(Sample)
flapro_rel_total_all

# save to file
write_tsv(flapro_rel_total_all %>% pivot_wider(names_from = feature, values_from = value), file.path(PROJ_OUTPUT_DIR, paste0("flapro_rel_total_all_", SCENARIO_COMPON, ".tsv")))

In [ ]:
flapro_rel[[SCENARIO_COMPON]] %>% 
    inner_join(meta_fla_clus, by = c("feature" = "FlaCluster")) %>%
    select(feature, Cluster_Pred) %>% 
    distinct() %>% 
    select(Cluster_Pred) %>% 
    table()

In [ ]:
# compute total *relative* fla counts per sample, stratified by Cluster_Pred
flapro_rel_total_by_Cluster_Pred = 
    flapro_rel[[SCENARIO_COMPON]] %>% 
    inner_join(meta_fla_clus, by = c("feature" = "FlaCluster")) %>%     
    group_by(Sample, Cluster_Pred) %>% 
    summarise(value = sum(value)) %>%    
    #mutate(feature = "sum_rel_abund") %>%
    mutate(feature = paste0("sum_rel_abund", "_F_", Cluster_Pred)) %>%    
    ungroup() %>%     
    select(-Cluster_Pred) %>%
    #inner_join(meta_samples, by = "Sample") %>% 
    arrange(Sample)    
flapro_rel_total_by_Cluster_Pred 

# save to file
write_tsv(flapro_rel_total_by_Cluster_Pred %>% pivot_wider(names_from = feature, values_from = value), file.path(PROJ_OUTPUT_DIR, paste0("flapro_rel_total_by_Cluster_Pred_", SCENARIO_COMPON, ".tsv")))

In [ ]:
flapro_rel_total_by_Cluster_Pred  %>% filter(value > 0)

flapro_rel_total_by_Cluster_Pred %>% inner_join(meta_samples, by = "Sample") %>% 
    select(Sample, Group) %>%
    distinct() %>%
    select(Group) %>%
    table()

In [ ]:
flapro_rel_total = rbind(flapro_rel_total_all, flapro_rel_total_by_Cluster_Pred)
flapro_rel_total

In [ ]:
samples_with_zero_Active = flapro_rel_total_by_Cluster_Pred %>% 
    filter(value == 0 & feature == "sum_rel_abund_F_active") %>%
    select(Sample)
samples_with_zero_Active

In [ ]:
# for each Sample, compute the ratio of Abundance_rel between Silent and Active
flapro_rel_Silent_Active_ratio = 
    flapro_rel_total_by_Cluster_Pred %>% 
    separate(feature, into = c("feature", "Cluster_Pred"), sep = "_F_") %>%
    # there are not more than a few samples with 0 Active rel abund, so just remove them
    anti_join(samples_with_zero_Active, by = "Sample") %>%
    group_by(Sample) %>%    
    mutate(ratio_silent_active = value[Cluster_Pred == "silent"] / value[Cluster_Pred == "active"]) %>%
    ungroup() %>%
    arrange(desc(ratio_silent_active)) %>% 
    select(-value, -Cluster_Pred) %>% 
    distinct() %>% 
    mutate(feature = "ratio_silent_active") %>% 
    rename(value = "ratio_silent_active")
flapro_rel_Silent_Active_ratio

# save to file
write_tsv(flapro_rel_Silent_Active_ratio %>% pivot_wider(names_from = feature, values_from = value), file.path(PROJ_OUTPUT_DIR, paste0("flapro_rel_Silent_Active_ratio_", SCENARIO_COMPON, ".tsv")))

In [ ]:
## for modeling, merge 2 columns into 1 to form a feature name
##flapro_rel_total_by_Cluster_Pred_for_lm = 
#flapro_rel_total_by_Cluster_Pred = flapro_rel_total_by_Cluster_Pred %>%
#    mutate(feature = paste0(feature, "_F_", Cluster_Pred)) %>% 
#    select(-Cluster_Pred)

### adjust features for Age - via residuals

In [ ]:
if(ADD_ADJUST_FOR_FACTORS_COMPON) {
    flapro_rel[[SCENARIO_COMPON]] = adjust_via_residuals(meta_samples, flapro_rel[[SCENARIO_COMPON]], ADD_ADJUST_FOR)
    flapro_rel_maj = adjust_via_residuals(meta_samples, flapro_rel_maj, ADD_ADJUST_FOR)
    flapro_rel_maj_log = adjust_via_residuals(meta_samples, flapro_rel_maj_log, ADD_ADJUST_FOR)
    flapro_rel_total = adjust_via_residuals(meta_samples, flapro_rel_total, ADD_ADJUST_FOR)
    if(SCENARIO_COMPON != "MTX_MGX_ratio") {
        flapro_alpha = adjust_via_residuals(meta_samples, flapro_alpha, ADD_ADJUST_FOR)
    }    
    flapro_rel_Silent_Active_ratio = adjust_via_residuals(meta_samples, flapro_rel_Silent_Active_ratio, ADD_ADJUST_FOR)    
}

In [ ]:
#flapro_rel_total_by_Cluster_Pred = flapro_rel_total_by_Cluster_Pred_for_lm %>%
#    separate(feature, into = c("feature", "Cluster_Pred"), sep = "_F_")

### viz summarized measures

In [ ]:
if(SCENARIO_COMPON != "MTX_MGX_ratio") {
    flapro_alpha_to_plot = flapro_alpha %>% separate(feature, into = c("feature", "Cluster_Pred"), sep = "_F_")
    #flapro_alpha_to_plot
}

In [ ]:
if(SCENARIO_COMPON != "MTX_MGX_ratio") {
    p1a = flapro_alpha_to_plot %>% 
        inner_join(meta_samples, by = "Sample") %>%
        filter(Cluster_Pred %in% c("all")) %>% 
        mutate(Cluster_Pred = factor(Cluster_Pred, levels = c("all", "active", "silent", "mixed", "not_defined"))) %>%
        ggplot(aes(x = Group, y = value, fill = Group)) +    
        ## log10 scale
        (if (!ADD_ADJUST_FOR_FACTORS_COMPON) scale_y_log10() else NULL) + 
        geom_violin() +
        geom_boxplot(width = 0.2, fill = "white") +
        geom_jitter(width = 0.3, height = 0, alpha = 0.2, size = 0.9, color = "#000000") + 
        theme_bw() +    
        facet_wrap( ~ feature, scales = "free_y") +
        scale_fill_manual(values = COHORT_COLORS) +
        # no grid
        theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank()) +        
        ylab("Flagellome alpha diversity") +
        # legend off
        theme(legend.position = "none") +
        (if (!REPEAT_MEAS_COMPON) 
        # Pairwise Wilcoxon test with significance annotations
        stat_compare_means(comparisons = STAT_PLOT_CMP,                        
                                    method = "wilcox.test", label = "p.format", 
                                    vjust = 0.1, size = 2, step.increase = 0.09)
        else NULL)

    p1b = flapro_alpha_to_plot %>% 
        inner_join(meta_samples, by = "Sample") %>%
        mutate(Cluster_Pred = factor(Cluster_Pred, levels = c("all", "active", "silent", "mixed", "not_defined"))) %>%
        ggplot(aes(x = Group, y = value, fill = Group)) +    
        ## log10 scale
        (if (!ADD_ADJUST_FOR_FACTORS_COMPON) scale_y_log10() else NULL) + 
        geom_violin() +
        geom_boxplot(width = 0.2, fill = "white") +
        geom_jitter(width = 0.3, height = 0, alpha = 0.2, size = 0.9, color = "#000000") + 
        theme_bw() +    
        facet_grid(Cluster_Pred ~ feature, scales = "free_y") +
        scale_fill_manual(values = COHORT_COLORS) +
        # no grid
        theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank()) +        
        ylab("Flagellome alpha diversity") +
        # legend off
        theme(legend.position = "none") +
        (if (!REPEAT_MEAS_COMPON) 
        # Pairwise Wilcoxon test with significance annotations
        stat_compare_means(comparisons = STAT_PLOT_CMP,                        
                                    method = "wilcox.test", label = "p.format", 
                                    vjust = 0.1, size = 2, step.increase = 0.09)
        else NULL)

    p1c = flapro_alpha_to_plot %>% 
        inner_join(meta_samples, by = "Sample") %>%
        filter(Cluster_Pred %in% c("all", "active", "silent")) %>% 
        mutate(Cluster_Pred = factor(Cluster_Pred, levels = c("all", "active", "silent", "mixed", "not_defined"))) %>%
        ggplot(aes(x = Group, y = value, fill = Group)) +    
        ## log10 scale
        (if (!ADD_ADJUST_FOR_FACTORS_COMPON) scale_y_log10() else NULL) + 
        geom_violin() +
        geom_boxplot(width = 0.2, fill = "white") +
        geom_jitter(width = 0.3, height = 0, alpha = 0.2, size = 0.9, color = "#000000") + 
        theme_bw() +    
        facet_grid(Cluster_Pred ~ feature, scales = "free_y") +
        scale_fill_manual(values = COHORT_COLORS) +
        # no grid
        theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank()) +        
        ylab("Flagellome alpha diversity") +
        # legend off
        theme(legend.position = "none") +
        (if (!REPEAT_MEAS_COMPON) 
        # Pairwise Wilcoxon test with significance annotations
        stat_compare_means(comparisons = STAT_PLOT_CMP,                        
                                    method = "wilcox.test", label = "p.format", 
                                    vjust = 0.1, size = 2, step.increase = 0.09)
        else NULL)        
}

In [ ]:
if(SCENARIO_COMPON != "MTX_MGX_ratio") {
    p.dims(8, 3)
    p = ggarrange(p1a, ncol = 1)
    plot(annotate_figure(p, top = text_grob(SCENARIO_COMPON)))
}

In [ ]:
if(SCENARIO_COMPON != "MTX_MGX_ratio") {
    p.dims(8, 15)
    p = ggarrange(p1b, ncol = 1)
    plot(annotate_figure(p, top = text_grob(SCENARIO_COMPON)))
}

In [ ]:
if(SCENARIO_COMPON != "MTX_MGX_ratio") {
    p.dims(8, 7.5)
    p = ggarrange(p1c, ncol = 1)
    plot(annotate_figure(p, top = text_grob(SCENARIO_COMPON)))
}

In [ ]:
flapro_rel_total_to_plot = flapro_rel_total %>% separate(feature, into = c("feature", "Cluster_Pred"), sep = "_F_")

In [ ]:
# compare total relative fla counts across the groups
p1 = flapro_rel_total_to_plot %>% inner_join(meta_samples, by = "Sample") %>%
    mutate(Cluster_Pred = factor(Cluster_Pred, levels = c("all", "active", "silent", "mixed", "not_defined"))) %>%
    ggplot(aes(x = Group, y = value, fill = Group)) +
    # log10 scale
    (if (!ADD_ADJUST_FOR_FACTORS_COMPON) scale_y_log10() else NULL) + 
    geom_violin() +
    geom_boxplot(width = 0.3, fill = "white") +
    geom_jitter(width = 0.3, height = 0, alpha = 0.2, size = 0.9, color = "#000000") +    
    theme_bw() +
    facet_wrap(~Cluster_Pred, ncol = 5) +
    scale_fill_manual(values = COHORT_COLORS) +
    # no grid
	theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank()) +
    # y axis title: Abundance
	ylab("Flagellome rel. abundance") +
    # legend off
    theme(legend.position = "none") +
    (if (!REPEAT_MEAS_COMPON) 
    # Pairwise Wilcoxon test with significance annotations
    stat_compare_means(comparisons = STAT_PLOT_CMP,                        
                                method = "wilcox.test", label = "p.format", 
                                vjust = 0.1, size = 2, step.increase = 0.07)
    else NULL)

p1b = flapro_rel_total_to_plot %>% inner_join(meta_samples, by = "Sample") %>%
    mutate(Cluster_Pred = factor(Cluster_Pred, levels = c("_all", "active", "silent", "mixed", "not_defined"))) %>%
    filter(Cluster_Pred %in% c("active", "silent")) %>% 
    ggplot(aes(x = Group, y = value, fill = Group)) +
    # log10 scale
    (if (!ADD_ADJUST_FOR_FACTORS_COMPON) scale_y_log10() else NULL) + 
    geom_violin() +
    geom_boxplot(width = 0.3, fill = "white") +
    geom_jitter(width = 0.3, height = 0, alpha = 0.2, size = 0.9, color = "#000000") +    
    theme_bw() +
    facet_wrap(~Cluster_Pred, ncol = 2) +
    scale_fill_manual(values = COHORT_COLORS) +
    # no grid
	theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank()) +
    # y axis title: Abundance
	ylab("Flagellome rel. abundance") +
    # legend off
    theme(legend.position = "none") +
    (if (!REPEAT_MEAS_COMPON) 
    # Pairwise Wilcoxon test with significance annotations
    stat_compare_means(comparisons = STAT_PLOT_CMP,                        
                                method = "wilcox.test", label = "p.format", 
                                vjust = 0.1, size = 2, step.increase = 0.07)
    else NULL)


In [ ]:
p.dims(5.2, 3)
p = ggarrange(p1b, ncol = 1)
annotate_figure(p, top = text_grob(SCENARIO_COMPON))

p.dims(12, 3)
p = ggarrange(p1, ncol = 1)
annotate_figure(p, top = text_grob(SCENARIO_COMPON))

In [ ]:
p1 = flapro_rel_Silent_Active_ratio %>% inner_join(meta_samples, by = "Sample") %>%
    ggplot(aes(x = Group, y = value, fill = Group)) +
    ## log10 scale
    (if (!ADD_ADJUST_FOR_FACTORS_COMPON) scale_y_log10() else NULL) + 
    geom_violin() +
    geom_boxplot(width = 0.2, fill = "white") +
    geom_jitter(width = 0.3, height = 0, alpha = 0.2, size = 0.9, color = "#000000") +    
    theme_bw() +    
    scale_fill_manual(values = COHORT_COLORS) +
    # no grid
	theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank()) +
	ylab("Silent/Active abundance ratio") +    
    # legend off
    theme(legend.position = "none") +
    (if (!REPEAT_MEAS_COMPON) 
    # Pairwise Wilcoxon test with significance annotations
    stat_compare_means(comparisons = STAT_PLOT_CMP,                        
                                method = "wilcox.test", label = "p.format", 
                                vjust = 0.1, size = 2, step.increase = 0.07)
    else NULL)


In [ ]:
p.dims(3.5, 3.5)
p = ggarrange(p1, ncol = 1)
annotate_figure(p, top = text_grob(SCENARIO_COMPON))

### Top Fla

In [ ]:
check_full(flapro_rel[[SCENARIO_COMPON]], "feature")

In [ ]:
# flapro_rel: boxplot the abundance per Fla in the order of median decreasing
top_n_fla = 
    flapro_rel[[SCENARIO_COMPON]] %>%
    mutate(mean_Abundance_rel = mean(value), .by = "feature") %>%    
    select(feature, mean_Abundance_rel) %>% 
    distinct() %>%         
    inner_join(meta_fla_clus, by = c("feature" = "FlaCluster")) %>%    
    arrange(desc(mean_Abundance_rel)) %>%
    filter(mean_Abundance_rel > 0) %>%
    head(200) 
top_n_fla

In [ ]:
p.dims(8, 12)
flapro_rel[[SCENARIO_COMPON]] %>% 
    inner_join(top_n_fla, by = "feature") %>%
    
    #ggplot(aes(x = reorder(feature, mean_Abundance_rel), y = value, color = Cluster_Exp)) +    
    #geom_jitter(aes(color = Cluster_Exp), size = 0.5, alpha = 0.4, width = 0.05) +    
    #scale_color_manual(values = FLA_CLASSES_COLORS) +    

    #ggplot(aes(x = reorder(feature, mean_Abundance_rel), y = value, shape = Cluster_Exp)) +    
    #geom_jitter(aes(shape = Cluster_Exp), size = 1, alpha = 0.4, width = 0.05, height = 0) +    
    #scale_shape_manual(values = FLA_CLASSES_SHAPES) +    

    ggplot(aes(x = reorder(feature, mean_Abundance_rel), y = value, shape = Cluster_Exp, color = Cluster_Exp)) +    
    geom_jitter(aes(shape = Cluster_Exp), size = 1, alpha = 0.4, width = 0.05, height = 0) +    
    scale_shape_manual(values = FLA_CLASSES_SHAPES) + 
    scale_color_manual(values = FLA_CLASSES_COLORS) +   

    theme_bw() +    
    theme(panel.border = element_rect(size = 0.1, colour = "black")) +
    # disable grid
    theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank()) +
    # disable x axis text    
    theme(axis.text.y = element_blank()) + 
    # disable y ticks
    theme(axis.ticks.y = element_blank()) +
    coord_flip() +
    labs(x = "Flagellin cluster", y = "Relative abundance") +
    # increase size of dot in legend
    guides(color = guide_legend(override.aes = list(size = 5))) +
    # log scale for y
    (if (!ADD_ADJUST_FOR_FACTORS_COMPON) scale_y_log10() else NULL)

In [ ]:
p.dims(8, 12)
flapro_rel[[SCENARIO_COMPON]] %>% 
    inner_join(top_n_fla, by = "feature") %>%

    #ggplot(aes(x = reorder(feature, mean_Abundance_rel), y = value, color = Cluster_Pred)) +    
    #geom_jitter(aes(color = Cluster_Pred), size = 0.3, alpha = 0.4, width = 0.05) +    
    #scale_color_manual(values = FLA_CLASSES_COLORS) +    

    ggplot(aes(x = reorder(feature, mean_Abundance_rel), y = value, shape = Cluster_Pred, color = Cluster_Pred)) +    
    geom_jitter(aes(shape = Cluster_Pred), size = 1, alpha = 0.4, width = 0.05, height = 0) +    
    scale_shape_manual(values = FLA_CLASSES_SHAPES) + 
    scale_color_manual(values = FLA_CLASSES_COLORS) +   

    theme_bw() +    
    theme(panel.border = element_rect(size = 0.1, colour = "black")) +
    # disable grid
    theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank()) +
    # disable x axis text    
    theme(axis.text.y = element_blank()) + 
    # disable y ticks
    theme(axis.ticks.y = element_blank()) +
    coord_flip() +
    labs(x = "Flagellin cluster", y = "Relative abundance") +
    # increase size of dot in legend
    guides(color = guide_legend(override.aes = list(size = 5))) +
    # log scale for y
    (if (!ADD_ADJUST_FOR_FACTORS_COMPON) scale_y_log10() else NULL)

### Beta diversity

In [ ]:
flapro_rel_df = 
    flapro_rel[[SCENARIO_COMPON]] %>%          
    select(feature, Sample, value) %>% 
    pivot_wider(names_from = "feature", values_from = "value", values_fill = 0) %>% 
    column_to_rownames("Sample") %>% 
    as.data.frame
flapro_rel_df

In [ ]:
# compute an Euclidean distance matrix from flapro_rel_df
dist_my_rel = dist(flapro_rel_df, method = "euclidean")

In [ ]:
# identify outliers based on the distance matrix
dist_my_rel_mat = as.matrix(dist_my_rel)
# Compute mean or median distance for each sample
mean_distances = rowMeans(dist_my_rel_mat)
median_distances = apply(dist_my_rel_mat, 1, median)

# Identify outliers using a threshold (e.g., X standard deviations from the mean)
threshold = mean(mean_distances) + REL_N_SIGMA_REL_OUTLIERS * sd(mean_distances)
outliers = which(mean_distances > threshold)

# Results
sort(names(outliers))
length(outliers)

In [ ]:
# remove outliers from dist_my_rel, a dist object
if(length(outliers)) {
    dist_my_rel_mat = dist_my_rel_mat[-outliers, -outliers]
    dist_my_rel = as.dist(dist_my_rel_mat)
    dim(dist_my_rel)
    flapro_rel_df_noOut = flapro_rel_df[-outliers,]
} else {
    flapro_rel_df_noOut = flapro_rel_df
}
dim(flapro_rel_df_noOut)

### PCoA biplot (rel.)

In [ ]:
# do PCoA
pcoa_res_rel = ape::pcoa(dist_my_rel)

# get the percentage of variance explained by top PCs
pve = round(pcoa_res_rel$values$Relative_eig[1:3] * 100, 1) 
# concatenate those values to PC1 and PC2
pve = paste0("PC", 1:3, " (", pve, "%)")
names(pve) = paste0("PC", 1:3)

# Extract PCoA coordinates for plotting
pcoa_scores = data.frame(pcoa_res_rel$vectors)
# rename all columns of pcoa_scores to PC1, PC2, PC3 and so on
colnames(pcoa_scores) = gsub(colnames(pcoa_scores), pattern = "Axis.", replacement = "PC")
#pcoa_scores

# Generalized computation of feature contributions for 3 PCs
num_pcs = 3  # Number of PCs to consider
feature_contributions = as.data.frame(
  do.call(cbind, lapply(1:num_pcs, function(pc) {
    apply(flapro_rel_df_noOut, 2, function(feature) suppressWarnings(cor(feature, pcoa_scores[[paste0("PC", pc)]])))
  }))
)
colnames(feature_contributions) = paste0("PC", 1:num_pcs)  # Name the columns
feature_contributions$Feature = colnames(flapro_rel_df_noOut)  # Add feature names

# Scale the feature contributions for visualization (optional)
feature_contributions = feature_contributions %>%
  mutate(across(starts_with("PC"), ~ .x * BIPLOT_ARROW_SCALING[[SCENARIO_COMPON]]))

# Compute magnitude of contributions and retain the top 5 features
feature_contributions = feature_contributions %>%
  rowwise() %>%
  mutate(Magnitude = sqrt(sum(c_across(starts_with("PC"))^2))) %>%
  ungroup() %>%
  arrange(desc(Magnitude)) %>%
  slice(1:REL_N_FEATURES_BIPLOT)  # Keep top features
feature_contributions

pcoa_vis = pcoa_scores %>% rownames_to_column("Sample") %>% 
    inner_join(meta_samples, by = "Sample")
#pcoa_vis

In [ ]:
# join meta to the feature_contributions
feature_contributions = feature_contributions %>% 
    inner_join(meta_fla_clus, by = c("Feature" = "FlaCluster")) %>%     
    mutate(Cluster_Species_trimmed = ifelse(		
			str_detect(Cluster_Species, ";"),			
			paste0(str_extract(Cluster_Species, "^[^;]+"), "+"),
			Cluster_Species)		
	) %>%	
    mutate(Feature_ext = paste(Feature, "\n", Cluster_Species_trimmed, sep = ""))  %>% 
	select(-Cluster_Species, -Cluster_Species_trimmed)
feature_contributions

In [ ]:
# biplot function
create_pcoa_biplot = function(pcoa_vis, feature_contributions, x_axis, y_axis, pve, color_var, scale_color = scale_color_manual(values = COHORT_COLORS)) {
  # Ensure the axis names are valid
  if (!(x_axis %in% colnames(pcoa_vis)) | !(y_axis %in% colnames(pcoa_vis))) {
    stop("Specified x_axis or y_axis is not in the pcoa_vis data.")
  }
  if (!(x_axis %in% colnames(feature_contributions)) | !(y_axis %in% colnames(feature_contributions))) {
    stop("Specified x_axis or y_axis is not in the feature_contributions data.")
  }

  # Create biplot
  ggplot() +
    # Sample points
    geom_point(data = pcoa_vis, aes_string(x = x_axis, y = y_axis, color = color_var), size = 0.5) +    

    ## Sample labels
    (if (BIPLOT_LABELS_SAMPLES) geom_text_repel(data = pcoa_vis, aes_string(x = x_axis, y = y_axis, label = "Sample"), size = 2, color = "#888888", segment.size = 0.1) else NULL) + 
        
    # X and Y labels for explained variance
    xlab(pve[x_axis]) +
    ylab(pve[y_axis]) +   
    # Theme and color scale
    theme_classic() + 
    scale_color +    
    # Increase bullet size in legend
    guides(color = guide_legend(override.aes = list(size = 4))) +

   # Feature arrows
    geom_segment(
      data = feature_contributions, 
      aes_string(x = 0, y = 0, xend = x_axis, yend = y_axis , 
        linetype = "Cluster_Pred"
        ), 
      arrow = arrow(length = unit(0.2, "cm"), type = "closed", angle = 15),      
      size = 0.2, color = "#888888"
    ) +
    # Feature labels
    geom_text(
      data = feature_contributions, 
      aes_string(x = x_axis, y = y_axis, label = "Feature_ext"), 
      vjust = "center", hjust = "middle", color = "black", size = 2
    ) +
    scale_linetype_manual(
      values = c("active" = "solid", "mixed" = "longdash", "silent" = "dashed", "not_defined" = "dotted"),
      name = "Cluster_Pred"
    ) 
}

p.dims(15, 8)
#p.dims(5, 10)
# PC1-2:
p1 = create_pcoa_biplot(pcoa_vis, feature_contributions, "PC1", "PC2", pve, "Group")
# PC1-3:
p2 = create_pcoa_biplot(pcoa_vis, feature_contributions, "PC1", "PC3", pve, "Group")

# plot them next to each other
p = ggarrange(p1, p2, ncol = 2, common.legend = TRUE, legend = "bottom")
annotate_figure(p, top = text_grob(SCENARIO_COMPON))

### Stat: summarized Fla features

In [ ]:
# init a list to store the tables to go to a multisheet xlsx
df_list_stats_compon_results = list()

In [ ]:
lm_in = rbind(        
        flapro_rel_total,                
        flapro_rel_Silent_Active_ratio) %>%
    inner_join(meta_samples, by = "Sample") %>%
    filter(Group %in% GROUPS_TO_COMPARE)
lm_in

In [ ]:
if(REPEAT_MEAS_COMPON) {
    lm_res = do_lmer_tidy(lm_in, rel_model_formula, arg_response_col = "value", arg_feature_col = "feature") %>% 
        select(-effect, -group)
} else {
    lm_res = do_lm_tidy(lm_in, rel_model_formula, arg_response_col = "value", arg_feature_col = "feature")
}
df.dims(50)    
lm_res #%>% filter(term == sel_factor_coef) %>% filter(p.value < 0.1)
df.dims(5)    

In [ ]:
df.dims(50)    
lm_res %>% filter(term == sel_factor_coef) %>% filter(p.value < 0.1)
df.dims(5)    

## save the results to file
tmp_sheet_name = paste0("SumFts_", SCENARIO_COMPON, "_", GROUPS_SUFFIX, "_", sel_factor_coef)
if(nchar(tmp_sheet_name) > 31) {
	print("Warning: xlsx sheet name too long! Reducing to 31 leading symbols.")
	tmp_sheet_name = substr(tmp_sheet_name, 0, 31)
	print(tmp_sheet_name)
}
df_list_stats_compon_results[[tmp_sheet_name]] = lm_res %>% filter(term == sel_factor_coef) %>% filter(p.value < 0.1)


### Stat: alpha diversity

In [ ]:
if(SCENARIO_COMPON != "MTX_MGX_ratio") {
    lm_in = flapro_alpha %>%
        inner_join(meta_samples, by = "Sample") %>%
        filter(Group %in% GROUPS_TO_COMPARE)
    lm_in
}

In [ ]:
if(SCENARIO_COMPON != "MTX_MGX_ratio") {
    if(REPEAT_MEAS_COMPON) {
        lm_res = do_lmer_tidy(lm_in, rel_model_formula, arg_response_col = "value", arg_feature_col = "feature") %>% 
        select(-effect, -group)
    } else {
        lm_res = do_lm_tidy(lm_in, rel_model_formula, arg_response_col = "value", arg_feature_col = "feature")
    }
    df.dims(50)    
    print(lm_res %>% filter(p.value < 0.1))
    df.dims(5)    

    # save the results to file
    tmp_sheet_name = paste0("FAlph_", SCENARIO_COMPON, "_", GROUPS_SUFFIX, "_", sel_factor_coef)
    if(nchar(tmp_sheet_name) > 31) {
        print("Warning: xlsx sheet name too long! Reducing to 31 leading symbols.")
        tmp_sheet_name = substr(tmp_sheet_name, 0, 31)
        print(tmp_sheet_name)
    }
    df_list_stats_compon_results[[tmp_sheet_name]] = lm_res %>% filter(p.value < 0.1)
}

### Stat: fla clusters (over flapro_rel_MAJ)

#### Linear model, all factors

In [ ]:
lm_in = flapro_rel_maj_log %>% 
    inner_join(meta_samples, by = "Sample") %>% 
    # subset to selected groups    
    filter(Group %in% GROUPS_TO_COMPARE)
lm_in

In [ ]:
if(REPEAT_MEAS_COMPON) {
    lm_res = do_lmer_tidy(lm_in, rel_model_formula, arg_response_col = "value", arg_feature_col = "feature") %>% 
        select(-effect, -group)
} else {
    lm_res = do_lm_tidy(lm_in, rel_model_formula, arg_response_col = "value", arg_feature_col = "feature")
}
df.dims(50)    
lm_res %>% filter(p.value < 0.1)
df.dims(5)    

In [ ]:
lm_res_sel_factor = lm_res %>% 
    filter(term == sel_factor_coef) %>%
    inner_join(meta_fla_clus, by = c("feature" = "FlaCluster")) %>% 
    mutate(p.adj = p.adjust(p.value, method = "fdr"))
df.dims(50) 
lm_res_sel_factor_s = lm_res_sel_factor %>% 
    ##filter(p.value < 0.1)
    filter(p.adj < 0.1) 
lm_res_sel_factor_s
df.dims(5)

In [ ]:
# save the results to file
tmp_sheet_name = paste0("FC_", SCENARIO_COMPON, "_", GROUPS_SUFFIX, "_", sel_factor_coef)
if(nchar(tmp_sheet_name) > 31) {
	print("Warning: xlsx sheet name too long! Reducing to 31 leading symbols.")
	tmp_sheet_name = substr(tmp_sheet_name, 0, 31)
	print(tmp_sheet_name)
}
df_list_stats_compon_results[[tmp_sheet_name]] = lm_res_sel_factor

In [ ]:
num_finds = nrow(lm_res_sel_factor_s)
num_finds
num_per_row = 5

my_w = min(13, 13 * (num_finds / num_per_row))
my_w
my_h = 3 * ceiling(num_finds / num_per_row)
my_h

p.dims(my_w, my_h) 
ttt = flapro_rel_maj_log %>% 	
	inner_join(lm_res_sel_factor_s, by = "feature") %>% 	
	inner_join(meta_samples, by = "Sample") %>% 
	mutate(sign_estimate = factor(sign(estimate))) %>% 		
	mutate(Cluster_Species_trimmed = ifelse(		
			str_detect(Cluster_Species, ";"),			
			paste0(str_extract(Cluster_Species, "^[^;]+"), "+"),
			Cluster_Species)		
	) %>%
	mutate(FlaCluster_ext = paste(feature, "\n", Cluster_Species_trimmed, "\n", Cluster_Pred, sep = "")) %>% 
    # subset to groups    
    filter(Group %in% GROUPS_TO_COMPARE)
#ttt

if(nrow(ttt) > 0) {
	ggplot(ttt, aes(y = value, x = Group)) +		
		## log scale y
		#scale_y_log10() +
		geom_boxplot(width = 0.1) +
		geom_violin(color="#888888", alpha=0.1, size=0.5) +
		geom_jitter(aes(color = sign_estimate), alpha=0.1, size=3, width=0.3, height = 0) +
		scale_color_manual(values = c("-1" = "blue", "1" = "red")) +
		facet_wrap(~FlaCluster_ext, ncol = num_per_row, scales = "free") +			
		theme_bw() +		
		# no grid
		theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank()) +
		# no grey fill for facet
		theme(strip.background = element_blank()) +
		# no legend
		theme(legend.position = "none") +

		# y axis title: Abundance
		ylab("Relative abundance (log10)")
}

#### Wilcoxon (only performed when there are 2 clinical groups selected)

In [ ]:
# Perform Wilcoxon test for each flagellin between the groups (without discarding the outliers)
wil_in = flapro_rel[[SCENARIO_COMPON]] %>% 
    inner_join(meta_samples %>% select(Sample, Group), by = "Sample") %>% 
    # subset to selected groups    
    filter(Group %in% GROUPS_TO_COMPARE)

#go_wilcox = length(GROUPS_TO_COMPARE) == 2
go_wilcox = FALSE

if(go_wilcox) {
    wilcox_results = wil_in %>%            
        group_by(FlaCluster) %>%
        summarise(
            p_value = wilcox.test(Abundance_rel ~ Group)$p.value,
            .groups = 'drop'
        ) %>% 
        arrange(p_value) %>% 
        # adjust p-values
        mutate(p_value_adj = p.adjust(p_value, method = "BH"))
    #wilcox_results

    wilcox_results = wilcox_results %>% 
        inner_join(meta_fla_clus, by = "FlaCluster") %>% 
        filter(p_value_adj < 0.1) %>% 
        #select(Flagellin_ID, Genus, Species, p_value, p_value_adj) %>% 
        arrange(p_value_adj)
}
if(go_wilcox) {
    df.dims(100)
    print(wilcox_results)
    df.dims(5)
}

In [ ]:
if(go_wilcox) {
	if(nrow(wilcox_results) > 0) {
		p.dims(6, 6)

		ttt = flapro_rel %>% 	
			semi_join(wilcox_results, by = "FlaCluster") %>% 	
			inner_join(meta_samples, by = "Sample") %>% 		
			# subset to groups
			filter(Group %in% GROUPS_TO_COMPARE)
		#ttt

		if(nrow(ttt) > 0) {
			ggplot(ttt, aes(y = Abundance_rel, x = Group)) +		
				# log scale y
				(if (!ADD_ADJUST_FOR_FACTORS_COMPON) scale_y_log10() else NULL) + 
				geom_boxplot(width = 0.1) +
				geom_violin(color="#888888", alpha=0.1, size=0.5) +
				geom_jitter(color="green", alpha=0.2, size=3, width=0.3) +
				facet_wrap(~FlaCluster) +	
				theme_bw()	
		}
	}
}

In [ ]:
# write the stat. results into a multi-sheet xls, with auto column width
openxlsx::write.xlsx(df_list_stats_compon_results, out_stat_compon_xlsx_file, colWidths="auto")

## Compositional analysis (available for MTX or MGX, not their ratio)

### Init normalized abundance and CLR 

In [ ]:
# prepare abundance data
abundance_my = cmultRepl2(t(countData))
clr_my = log(abundance_my) - rowMeans(log(abundance_my)) # CLR transformation
dim(abundance_my)

In [ ]:
clr_my_long = clr_my %>% rownames_to_column("Sample") %>%
    pivot_longer(cols = -c("Sample"), names_to = "feature", values_to = "value") 
clr_my_long

In [ ]:
# adjust compositional data for factor(s) - via residuals
if(ADD_ADJUST_FOR_FACTORS_NB) {
    clr_my_long = adjust_via_residuals(meta_samples, clr_my_long, ADD_ADJUST_FOR)
    clr_my = clr_my_long %>% pivot_wider(names_from = "feature", values_from = "value") %>% 
        column_to_rownames("Sample") %>% 
        as.data.frame()
    # adjust abundance_my accordingly
    abundance_my = inverse_clr(clr_my)
}

### PCoA (Aitchison)

In [ ]:
# compute an Euclidean distance matrix from clr_my
dist_my = dist(clr_my, method = "euclidean")

pcoa_res = ape::pcoa(dist_my)
pve = round(pcoa_res$values$Relative_eig[1:3] * 100, 1) 
# concatenate those values to PC1 and PC2
pve = paste0("PC", 1:3, " (", pve, "%)")

# plot a principal coordinate analysis
pcoa_my = cmdscale(dist_my, k = 3)
pcoa_my = as.data.frame(pcoa_my) %>% 
    rename(PCo1 = V1, PCo2 = V2, PCo3 = V3) #%>%
	#mutate(Sample = rownames(pcoa$points))
pcoa_my = cbind(pcoa_my, meta_samples_NB)

p.dims(9, 3)
# plot PC1-2 and PC2-3 next to each other
# PC1-2:
p1 = ggplot(pcoa_my, aes(x = PCo1, y = PCo2, color = Group)) + 
  geom_point(size = 2) +  
  xlab(pve[1]) +
  ylab(pve[2]) +   
  theme_classic() + 
  scale_color_manual(values = COHORT_COLORS) +
  # increase bullets size in legend
  guides(color = guide_legend(override.aes = list(size = 4)))
#p1

# PC1-3:
p2 = ggplot(pcoa_my, aes(x = PCo1, y = PCo3, color = Group)) + 
  geom_point(size = 2) +  
  xlab(pve[1]) +
  ylab(pve[3]) +   
  theme_classic() +  
  scale_color_manual(values = COHORT_COLORS) +
  # increase bullets size in legend
  guides(color = guide_legend(override.aes = list(size = 4)))
#p2
# plot p1 and p2 next to each other
gridExtra::grid.arrange(p1, p2, ncol=2)

### save a copy so that we can later compute the NB for the full set of samples

In [ ]:
countData_full = countData
meta_samples_full = meta_samples
abundance_my_full = abundance_my
clr_my_full = clr_my

### Create filter #2 for samples

In [ ]:
to_leave2 = meta_samples_NB %>% 	

	filter(Group %in% GROUPS_TO_COMPARE) %>% 

	select(Sample) %>% pull

length(to_leave2)

### generate a filtering preview (might visualize pre- and post-filtering data here)

In [ ]:
meta_samples_preview = meta_samples_NB %>% filter(Sample %in% to_leave2)
nrow(meta_samples_preview)

meta_samples_preview %>% select(Group) %>% table()

In [ ]:
length(to_leave2)
#setdiff(to_leave2, colnames(countData))

In [ ]:
countData_preview = countData[, sort(to_leave2)]

### apply the filter #2

In [ ]:
countData = countData_preview
meta_samples = meta_samples_preview
abundance_my = abundance_my[sort(colnames(countData)),] 
clr_my = clr_my[sort(colnames(countData_preview)),]


# meta_samples: drop the non-presented factor levels
meta_samples$Group = droplevels(meta_samples$Group)

In [ ]:
test_that("countData and meta match", {
	expect_equal(sort(meta_samples %>% select(Sample) %>% pull), sort(colnames(countData)))
})

### weiter

In [ ]:
# prepare a dataframe version of meta_by_sample and sort it by Sample
meta_df = data.frame(meta_samples %>% column_to_rownames("Sample")) 
#meta_df = meta_df[order(rownames(meta_df)),]
meta_df

In [ ]:
test_that("meta_df and matrices have the same samples in the same order", {
	expect_equal(rownames(meta_df), rownames(clr_my))	
})

In [ ]:
# though there was initial Fla filtering, some might have become zeroed out during the samples filtering
# so we need to remove them.
# get the names of the rows in countData with fewer than 5 non-zero values
# and remove them from countData and bracken_perc
low_features = names(which(rowSums(countData > 0) < 5))
test_that("no low features", {
	expect_equal(length(low_features), 0)
})

### Nearest Balance:

### PERMANOVA

In [ ]:
## check for heteroscedasticity across the factor of interest
#anova(betadisper(dist(clr_my), meta_df$sel_factor))

In [ ]:
df.dims(8)
adonis2(reformulate(init_permanova_formula, response = "dist(clr_my)"), meta_df, permutations = N_PERMANOVA, by = "terms")

In [ ]:
adonis2(reformulate(lm_nb_formula, response = "dist(clr_my)"), meta_df, permutations = N_PERMANOVA, by = "terms")

In [ ]:
coef_oa_bacs = do_clr_lm_get_fac_coeffs(lm_nb_formula, clr_my, meta_df, sel_factor_coef)
coef_oa_bacs

### Prepare splits for cross-validation

In [ ]:
if(!REPEAT_MEAS_NB) {
    # Option A: default way: balanced by the factor of interest
    splits = caret::createDataPartition(times = n_sim, y = meta_df[[sel_factor]], p = train_prop)
} else {
    # Option B: custom way: in each split, select 1 random time point per participant (and the test set is the rest - hence multiple time points - but we don't use it, so it's fine)
    splits = vector(mode = "list", n_sim)
    tmp_hashes = list()
    for(i in 1:n_sim) {    
        # the way below produces a single random time point per participant
        # different across the iterations
        subset_meta_df = meta_df %>% 
            mutate(rownum = row_number()) %>%
            dplyr::slice_sample(n = 1, by = !!sym(REPEAT_MEAS_FACTOR)) %>%
            dplyr::ungroup()
        cur_tmp_hash = digest::digest(subset_meta_df)
        #print(cur_tmp_hash)    
        tmp_hashes[[i]] = cur_tmp_hash
        #print(subset_meta_df)
        
        one_subset_split = caret::createDataPartition(times = 1, y = subset_meta_df[[sel_factor]], p = 1)   
        splits[[i]] = as.vector(unlist(subset_meta_df[unlist(one_subset_split), "rownum"]))
    }
    # test that the randomization really worked across the iterations
    test_that("randomization across iterations", {
        expect_equal(length(unique(tmp_hashes)), n_sim)
    })
    names(splits) = paste("Resample", 1:n_sim, sep = "")
}

### Parallel NB generation via cross-validations

In [ ]:
nb_list = parallel_run_nb(lm_nb_formula, sel_factor_coef, clr_my, meta_df, splits, n_sim, num_rparallel_cores)

In [ ]:
# in a repetitive measures case: PERMANOVA within a 1-sample-per-subject subsets 
# and then compare the generated p-values distribution to the significance threshold
if(REPEAT_MEAS_NB) {
    pvs_perm = c()
    for(i in 1:n_sim) {
        split = splits[[i]]    
        cur_perm = adonis2(reformulate(init_permanova_formula, response = "dist(clr_my[split,])"), meta_df[split,,drop=F], permutations = N_PERMANOVA, by = "terms")
        pvs_perm[i] = cur_perm[sel_factor, "Pr(>F)"]       
    }
    print(paste0("median p: ", median(pvs_perm)))
    print(paste0("sd p: ", sd(pvs_perm)))
    # compare the distribution with the significance threshold    
    print(wilcox.test(pvs_perm, mu = 0.1, alternative = "greater"))
}

### Process outputs of NB

In [ ]:
res = aggregate_balance_iterations(nb_list, reproducibility_threshold)
#res
sbp_iters = res$sbp_iters
sbp_consensus  = res$sbp_consensus

In [ ]:
nb = format_consensus_balance(sbp_consensus)
nb

In [ ]:
balance_size(nb)

In [ ]:
# compute the consensus balance values for each sample 
nb_vals = compute_balance(abundance_my, nb)
#nb_vals

# link metadata to the balance values
nb_vals_meta = nb_vals %>% inner_join(meta_samples, by = "Sample")
nb_vals_meta

### Selected taxa within the balance: check out

In [ ]:
df.dims(20)
sbp_iters %>% filter(taxName %like% "APC11219_1")
df.dims(5)

### Plot the balance values against the factors

In [ ]:
# Plot the relation between the balance values and continuous factor
p.dims(4,3)
ggplot(nb_vals_meta, aes(x = as.factor(!!sym(sel_factor)), y = NB_Value)) + 
	geom_violin() +
	geom_boxplot(width = 0.2) +
	geom_jitter(color="red", alpha=0.2, size=3, width=0.2) +
	#geom_point(color="red", alpha=0.2, size=3) +
	
	#geom_smooth(method = "lm", se = FALSE) +
	#geom_text(size = 1.5) +
	stat_smooth(method = "loess", se = FALSE) +
	labs(x = sel_factor, y = "Nearest balance") + theme(legend.position = "none") +	
	theme_bw()

In [ ]:
# Plot the relation between the balance values and continuous factor
p.dims(4,3)
ggplot(nb_vals_meta, aes(x = Group, y = NB_Value)) + 
	#geom_boxplot() +
    geom_violin(color="grey", alpha=0.7, size=0.5) +
	geom_boxplot(color="black", alpha=1, size=0.5, width = 0.3) +
	geom_jitter(color="red", alpha=0.3, size=2, width=0.2) +
	
	
	#geom_smooth(method = "lm", se = FALSE) +
	#geom_text(size = 1.5) +
	stat_smooth(method = "loess", se = FALSE) +
	labs(x = "Group", y = "Balance values") + theme(legend.position = "none") +	
	theme_bw()

In [ ]:
# do regression
nb_vals_meta %>% dplyr::do(broom::tidy(lm(reformulate(sel_factor, response = "NB_Value"), .)))


### go on preparing the balance

In [ ]:
# join sbp_consensus_reprod with coef_oa_bacs
nb_reprod_coef = join_coefs_and_sbp_reprod(get_sbp_consensus_with_reprod(sbp_iters), coef_oa_bacs)

df.dims(20)
nb_reprod_coef %>% arrange(desc(reprod))
df.dims(5)

In [ ]:
# add features info
nb_reprod_coef = nb_reprod_coef %>% 	
	inner_join(meta_fla_clus, by = c("taxName" = "FlaCluster")) %>% 
	mutate(Cluster_Species_trimmed = ifelse(		
			str_detect(Cluster_Species, ";"),			
			paste0(str_extract(Cluster_Species, "^[^;]+"), "+"),
			Cluster_Species)		
	) %>%
	mutate(taxName_ext = paste(taxName, Cluster_Species_trimmed, sep = "; "))
nb_reprod_coef

### Coefficients for each taxon in NB: plot

In [ ]:
p.dims(10,5)
# draw horizontal barplots of the coefficient for each taxName
pic = 
ggplot(nb_reprod_coef, aes(x = lm_coef, y = reorder(taxName_ext, lm_coef), fill = Cluster_Exp)) +	
	#geom_bar(stat = "identity", linewidth = 0.2, position = "dodge") +
	geom_bar(stat = "identity", linewidth = 0.2) +
	theme_bw() +
	scale_alpha_continuous(range = c(0.1, 1)) +	
	theme(axis.text = element_text(size = 6)) +	
	labs(x = "CLR LM coefficient", y = "") + 	
	# rotate labels
	theme(axis.text.x = element_text(angle = 60, hjust = 1), 
		plot.margin = margin(l = 20)) +
	# rotate the plot 90 degrees
	coord_flip() + 
	# set a palette for the fill
	scale_fill_simpsons() +	
	#scale_fill_ucscgb() +
	#scale_fill_igv() +
	
	#scale_fill_manual(values = all_fams_pal) +

	# legend text 10 
	theme(legend.text = element_text(size = 8))
pic

In [ ]:
#p.dims(10,5)
p.dims(8, 5)
# draw horizontal barplots of the coefficient for each taxName
pic = 
ggplot(nb_reprod_coef, aes(x = lm_coef, y = reorder(taxName_ext, lm_coef), fill = Cluster_Pred)) +	
#ggplot(nb_reprod_coef, aes(x = lm_coef, y = reorder(taxName, lm_coef), fill = Family)) +	
#ggplot(nb_reprod_coef, aes(x = lm_coef, y = reorder(taxName, lm_coef), fill = Experimental)) +	# none?..
	#geom_bar(stat = "identity", linewidth = 0.2, position = "dodge") +
	geom_bar(stat = "identity", linewidth = 0.2) +
	theme_bw() +
	scale_alpha_continuous(range = c(0.1, 1)) +	
	theme(axis.text = element_text(size = 6)) +	
	labs(x = "CLR LM coefficient", y = "") + 	
	# rotate labels
	theme(axis.text.x = element_text(angle = 60, hjust = 1), 
		plot.margin = margin(l = 20)) +
	# rotate the plot 90 degrees
	coord_flip() + 
	# set a palette for the fill
	#scale_fill_simpsons() +	
	#scale_fill_ucscgb() +
	#scale_fill_igv() +
	scale_fill_manual(values = FLA_CLASSES_COLORS) +
	
	#scale_fill_manual(values = all_fams_pal) +

	# legend text 10 
	theme(legend.text = element_text(size = 8))
pic


In [ ]:
p.dims(9,5)
# draw horizontal barplots of the coefficient for each taxName
pic = 
ggplot(nb_reprod_coef, aes(x = lm_coef, y = reorder(taxName_ext, lm_coef), fill = Cluster_Family)) +	
	#geom_bar(stat = "identity", linewidth = 0.2, position = "dodge") +
	geom_bar(stat = "identity", linewidth = 0.2) +
	theme_bw() +	
	labs(x = "Taxon coefficient", y = "") + 	
	coord_flip() + 
	
	theme(
			## legend (text), OR		
			legend.text = element_text(size = 10), legend.position = "right",
			#legend.text = element_text(size = 10), legend.position = "bottom",
			# legend OFF
			#legend.position = "none",

			#panel.border = element_blank(),
			panel.grid.major = element_blank(),
			panel.grid.minor = element_blank(),
			axis.line = element_line(colour = "black", linewidth = 0),
			axis.ticks = element_line(colour = "black", linewidth = 0.1),
			plot.title = element_text(hjust = 0.5, face = "bold"),

			strip.background = element_blank(),
			strip.text.x = element_blank(),
			panel.border = element_rect(colour = "black", linewidth = 1),

			#axis.text = element_text(size = 10),

			axis.text.x = element_text(size = 8, color = "black",
				## enable tilted labels
				# default:
				angle = 60, 
				# for MAGs (labels are longer)
				#angle = 70, 
				hjust = 1), 
			plot.margin = margin(r = 5, l = 40),

			# or disable labels
			#axis.text.x = element_blank(), 
			#plot.margin = margin(l = 20)			

			axis.text.y = element_text(size = 10, color = "black"),
			axis.title.x = element_text(size = 12, face = "bold", color = "black"),
			axis.title.y = element_text(size = 12, face = "bold", color = "black")			
		) +

	scale_fill_igv()	
	
pic


In [ ]:
fig_maker = function(fac, prof, tax_level) {

	pic = ggplot(list_nb_reprod_coef %>% filter(factor == fac, profiling == prof, taxonomic_level == tax_level), aes(x = lm_coef, y = reorder(taxName_ext, lm_coef), fill = Family)) +			
		#geom_bar(stat = "identity", width = 1) +
		geom_bar(stat = "identity") +

		theme_bw() +		
		
		theme(axis.text = element_text(size = 10)) +	
		labs(x = "Taxon coefficient", y = "") + 	
		# rotate labels
		theme(
			## enable tilted labels
			axis.text.x = element_text(size = 7, 

				# default:
				#angle = 60, 
				# for MAGs (labels are longer)
				angle = 70, 

				hjust = 1), 
			plot.margin = margin(l = 40)

			# or disable labels
			#axis.text.x = element_blank(), 
			#plot.margin = margin(l = 20)
			) +

		# This line can be toggled on/off:			
		ggtitle(qfactors_plots_n_tables_inv[[fac]]) +				

		# rotate the plot 90 degrees
		coord_flip() + 

		# set a palette for the fill		
		#scale_fill_manual(values = all_fams_pal) +
		scale_fill_manual(values = all_fams_pal, breaks = names(all_fams_pal)) +

		## legend (text), OR		
		#theme(legend.text = element_text(size = 8), legend.position = "right") +
		#theme(legend.text = element_text(size = 8), legend.position = "bottom") +
		# legend OFF
		theme(legend.position = "none") +

		theme(
			#panel.border = element_blank(),
			panel.grid.major = element_blank(),
			panel.grid.minor = element_blank(),
			axis.line = element_line(colour = "black", linewidth = 0),
			axis.ticks = element_line(colour = "black", linewidth = 0.1),
			plot.title = element_text(hjust = 0.5, face = "bold"),

			strip.background = element_blank(),
			strip.text.x = element_blank(),
			panel.border = element_rect(colour = "black", linewidth = 1),

			axis.text.x = element_text(size = 8, color = "black"),
			axis.text.y = element_text(size = 9, face = "bold", color = "black"),
			axis.title.x = element_text(size = 10, face = "bold", color = "black"),
			axis.title.y = element_text(size = 10, face = "bold", color = "black")
			
			)
	
	#pic
}

In [ ]:
p.dims(10, 5)

ttt = flapro_rel[[SCENARIO_NB]] %>% 
	inner_join(nb_reprod_coef, by = c("feature" = "taxName")) %>% 
	select(feature, Sample, value, b1, Cluster_Species, Cluster_Pred) %>%
	inner_join(meta_samples_NB, by = "Sample")  %>% 
	mutate(Association = ifelse(b1 > 0, "positive", "negative")) %>% 

	mutate(Cluster_Species_trimmed = ifelse(		
			str_detect(Cluster_Species, ";"),			
			paste0(str_extract(Cluster_Species, "^[^;]+"), "+"),
			Cluster_Species)		
	) %>%
	mutate(FlaCluster_ext = paste(feature, "\n", Cluster_Species_trimmed, "\n", Cluster_Pred, sep = ""))	
	#mutate(FlaCluster_ext = paste(FlaCluster, "\n", Cluster_Species, "\n", Cluster_Pred_v3, sep = "")) 
ttt

ggplot(ttt, aes(y = value, x = Group, color = Association)) +		
	# log scale y
	(if (!ADD_ADJUST_FOR_FACTORS_COMPON) scale_y_log10() else NULL) + 
	labs(y = "Abundance (log10)") +	
	geom_boxplot(width = 0.1) +
	geom_violin(color="#888888", alpha=0.1, size=0.5) +
	geom_jitter(alpha=0.1, size=3, width=0.3) +
	facet_wrap(~FlaCluster_ext, ncol = 4) +	
	scale_color_manual(values = c("negative" = "blue", "positive" = "red")) +
	theme_bw()	

### Compute the NB for the full dataset
Meaningful if the above analysis was conducted for a subset of the samples.

In [ ]:
# compute the consensus balance values for each sample 
nb_vals_full = compute_balance(abundance_my_full, nb)
dim(nb_vals_full)
nb_vals_meta_full = nb_vals_full %>% inner_join(meta_samples_full, by = "Sample")
nb_vals_meta_full

In [ ]:
p.dims(8, 4)
ggplot(nb_vals_meta_full, aes(x = as.factor(!!sym(sel_factor)), y = NB_Value)) + 
    geom_boxplot(width = 0.3) +
    geom_jitter(aes(color = as.factor(!!sym(sel_factor))), alpha=0.7, size=2, width=0.3, height = 0) +
    #facet_grid(. ~ !!sym(sel_factor)) +
    #geom_point(color="red", alpha=0.2, size=3) +
    #geom_smooth(method = "lm", se = FALSE) +	
    #geom_text(size = 1.5) +
    #stat_smooth(method = "loess", se = FALSE) +
    labs(x = sel_factor, y = "Nearest balance") + theme(legend.position = "none") +	
    scale_color_futurama() +	
    
    # y tick and y axis label every 1
    scale_y_continuous(breaks = scales::pretty_breaks(n = 10)) +	
    theme_bw()

# Per-subject balance

In [ ]:
to_plot = nb_vals_meta_full %>% 
        mutate(Participant_ID = factor(Participant_ID, 
            levels = sample(unique(Participant_ID[order(Group)])))) %>% 
        mutate(sample_num_seq = paste0(Group, "_", Participant_ID, "_", week_num)) 
to_plot

In [ ]:
p.dims(14, 14)
ggplot(to_plot,        
        #aes(x = as.factor(!!sym(sel_factor)), y = NB_Value)) + 
        aes(x = sample_num_seq, y = NB_Value)) +        
    geom_point(aes(color = Participant_ID), alpha=0.7, size=2, width=0.3, height = 0) +    
    facet_wrap(~ Group, nrow = 3, scales = "free" ) +
    #geom_point(color="red", alpha=0.2, size=3) +
    #geom_smooth(method = "lm", se = FALSE) +	
    #geom_text(size = 1.5) +
    #stat_smooth(method = "loess", se = FALSE) +
    labs(x = sel_factor, y = "Nearest balance") + theme(legend.position = "none") +	
    # shuffle the default ggplot palette
    


    #scale_color_futurama() +	
    theme_bw() +
    # no grid
    theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank()) +
    # no X axis labels
    theme(axis.text.x = element_blank())    
    

In [ ]:
p.dims(12, 8)
ggplot(to_plot,        
        #aes(x = as.factor(!!sym(sel_factor)), y = NB_Value)) + 
        aes(x = week_num, y = NB_Value, group = Participant_ID)) +            
    geom_line(alpha=0.7, size=0.3, height = 0) +    
    facet_wrap(~ Group, nrow = 3, scales = "free" ) +
    geom_point(color="red", alpha=0.6, size=2) +
    #geom_smooth(method = "lm", se = FALSE) +	
    #geom_text(size = 1.5) +
    #stat_smooth(method = "loess", se = FALSE) +
    labs(x = sel_factor, y = "Nearest balance") + theme(legend.position = "none") +	
    #scale_color_futurama() +	
    theme_bw() +
    # no grid
    theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank())